In [ ]:
pip install pandas geopandas matplotlib

In [ ]:
!pip install contextily

## Loader

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print('Uploaded file:', filename)

# Get the file path of the uploaded file
# file_path = list(uploaded.keys())[0]

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# Load each shapefile
buildings = gpd.read_file('shape/buildings.shp')
landuse = gpd.read_file('shape/landuse.shp')
natural = gpd.read_file('shape/natural.shp')
places = gpd.read_file('shape/places.shp')
points = gpd.read_file('shape/points.shp')
railways = gpd.read_file('shape/railways.shp')
roads = gpd.read_file('shape/roads.shp')
waterways = gpd.read_file('shape/waterways.shp')

In [ ]:
#buildings

In [ ]:
#landuse

In [ ]:
#natural

In [ ]:
#places

In [ ]:
#points

In [ ]:
#railways

In [ ]:
#roads

In [ ]:
#waterways

In [ ]:
import numpy as np

# Plot each layer
fig, ax = plt.subplots(figsize=(100, 100))

# Set background color
ax.set_facecolor('#EEEEEE')  # Set to light gray, you can use any color you want

buildings.plot(ax=ax, color='#FFFFFF')
landuse.plot(ax=ax, color='#28B463')
natural.plot(ax=ax, color='#F8C471')
places.plot(ax=ax, color='#99A3A4')
points.plot(ax=ax, color='#48C9B0')
railways.plot(ax=ax, color='#4A235A')
roads.plot(ax=ax, color='#000000')
waterways.plot(ax=ax, color='#85C1E9')

# Get the bounding box of the GeoDataFrame
minx, miny, maxx, maxy = buildings.total_bounds

# Print the bounding box to verify values
# print("Bounding box:", minx, miny, maxx, maxy)

# Generate random coordinates within the bounding box of your data
num_points = 1000  # Number of random points to generate
random_x = np.random.uniform(minx, maxx, num_points)
random_y = np.random.uniform(miny, maxy, num_points)

# Plot the random points
ax.scatter(random_x, random_y, color='red', marker='x', label='Random Points')

# Show the plot
plt.show()

In [ ]:
city_zones = buildings

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

class NearestNeighborClassifierManual:
    def __init__(self):
        self.X_train = None
        self.y_train = None

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        preds = []
        #To Do
        for x_test in X_test:
            dist = np.sum((self.X_train - x_test) ** 2, axis=1)
            dist = np.sqrt(dist)
            nnidx = np.argmin(dist)
            preds.append(self.y_train[nnidx])
        return np.array(preds)

#Classifier
classifier = NearestNeighborClassifierManual()
data = np.column_stack((random_x, random_y))
labels = np.random.randint(0, 2, 1000)
classifier.fit(data, labels)

predictions = classifier.predict(data)
fig, ax = plt.subplots(figsize=(100, 100))

#Plot Map
ax.set_facecolor('#EEEEEE') 

buildings.plot(ax=ax, color='#FFFFFF')
landuse.plot(ax=ax, color='#28B463')
natural.plot(ax=ax, color='#F8C471')
places.plot(ax=ax, color='#99A3A4')
points.plot(ax=ax, color='#48C9B0')
railways.plot(ax=ax, color='#4A235A')
roads.plot(ax=ax, color='#000000')
waterways.plot(ax=ax, color='#85C1E9')

ax.scatter(random_x, random_y, color='red', marker='x', label='Random Points')

#Show Circles for clusters
clusters = {}
for i, cluster_label in enumerate(predictions):
    if cluster_label not in clusters:
        clusters[cluster_label] = []
    clusters[cluster_label].append((random_x[i], random_y[i]))

for cluster_points in clusters.values():
    cluster_points = np.array(cluster_points)
    centroid = cluster_points.mean(axis=0)
    distances = np.linalg.norm(cluster_points - centroid, axis=1)
    radius = distances.max()
    circle = plt.Circle(centroid, radius, color='r', fill=False)
    ax.add_artist(circle)

ax.legend(['Buildings', 'Land Use', 'Natural', 'Places', 'Points', 'Railways', 'Roads', 'Waterways', 'Random Points', 'Clusters'])

plt.show()


### Help Codes for referance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

# Generate random points
np.random.seed(0)
num_points = 100
points_x = np.random.uniform(low=0, high=10, size=num_points)
points_y = np.random.uniform(low=0, high=10, size=num_points)

# Plot random points
plt.figure(figsize=(8, 6))
plt.scatter(points_x, points_y, color='blue', label='Random Points')

# Simulate epidemic circles
epidemic_centers = [(3, 6), (8, 2)]  # Coordinates of epidemic centers
epidemic_radii = [2, 1.5]  # Radii of epidemic circles

# Create labels for points (0 for outside the circles, 1 for inside)
labels = np.zeros(num_points)
for i, (x, y) in enumerate(zip(points_x, points_y)):
    for center, radius in zip(epidemic_centers, epidemic_radii):
        if (x - center[0])**2 + (y - center[1])**2 <= radius**2:
            labels[i] = 1
            break

# Use Kernel Density Estimation (KDE) to estimate intensity of zones
X = np.column_stack((points_x, points_y))
kde = KernelDensity(bandwidth=0.5, kernel='gaussian')
kde.fit(X)

# Create grid for heatmap
x_grid, y_grid = np.meshgrid(np.linspace(0, 10, 100), np.linspace(0, 10, 100))
grid_points = np.vstack([x_grid.ravel(), y_grid.ravel()]).T
densities = np.exp(kde.score_samples(grid_points))
densities = densities.reshape(x_grid.shape)

# Plot heatmap
cmap = plt.cm.OrRd  # Choose colormap
norm = Normalize(vmin=densities.min(), vmax=densities.max())
plt.contourf(x_grid, y_grid, densities, cmap=cmap, norm=norm, alpha=0.5)

# Plot epidemic circles
for center, radius in zip(epidemic_centers, epidemic_radii):
    circle = plt.Circle(center, radius, color='red', alpha=0.3)
    plt.gca().add_patch(circle)

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Epidemic Circle Zones with Intensity')
plt.colorbar(ScalarMappable(norm=norm, cmap=cmap), label='Intensity')
plt.legend()
plt.axis('equal')
plt.show()


In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.cm as cm

# Combine random_x and random_y into a single array
random_points = np.column_stack((random_x, random_y))

# Define the number of nearest neighbors for the KNN model
k = 1

# Create and fit KNN model
knn_model = NearestNeighbors(n_neighbors=k)
knn_model.fit(random_points)

# Find nearest neighbors and distances for each point
distances, indices = knn_model.kneighbors(random_points)

# Calculate radius based on the distance to the nearest neighbor
epidemic_radii = distances.flatten()

# Plot circles representing epidemic centers with color intensity dependent on radius
fig, ax = plt.subplots(figsize=(10, 10))

# Plot random points
ax.scatter(random_x, random_y, color='blue', label='Random Points')

# Plot circles representing epidemic centers with color intensity dependent on radius
for i, (x, y) in enumerate(random_points):
    circle = plt.Circle((x, y), epidemic_radii[i], color=cm.Blues(epidemic_radii[i] / np.max(epidemic_radii)), alpha=0.5)
    ax.add_artist(circle)

# Add legend
ax.legend()

# Show the plot
plt.show()
